## Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import csv
from datetime import datetime
import numpy as np
from config import db_user, db_password, db_host, db_port, db_name

## Extract csv into DataFrame

In [2]:
csv_file = "data/US_Accidents_Dec20_Updated.csv"
accident_data_df = pd.read_csv(csv_file)
accident_data_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,37.165850,-121.988062,1.4,Stationary traffic on CA-17 from Summit Rd (CA...,...,False,False,False,False,False,False,Night,Night,Night,Night
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,39.110390,-119.773781,0.0,Accident on US-395 Southbound at Topsy Ln.,...,False,False,False,False,True,False,Day,Day,Day,Day
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,26.102942,-80.265091,0.0,Accident on I-595 Westbound at Exit 4 / Pine I...,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
accident_data_df['date'] = pd.to_datetime(accident_data_df['Start_Time']).dt.date
accident_data_df['time'] = pd.to_datetime(accident_data_df['Start_Time']).dt.time
accident_data_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,date,time
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,...,False,False,False,False,Day,Day,Day,Day,2019-05-21,08:29:55
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,...,False,False,False,False,Day,Day,Day,Day,2019-10-07,17:43:09
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,37.165850,-121.988062,1.4,Stationary traffic on CA-17 from Summit Rd (CA...,...,False,False,False,False,Night,Night,Night,Night,2020-12-13,21:53:00
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,39.110390,-119.773781,0.0,Accident on US-395 Southbound at Topsy Ln.,...,False,False,True,False,Day,Day,Day,Day,2018-04-17,16:51:23
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,26.102942,-80.265091,0.0,Accident on I-595 Westbound at Exit 4 / Pine I...,...,False,False,True,False,Day,Day,Day,Day,2016-08-31,17:40:49


In [4]:
csv_file2 = "data/WeatherEvents_Jan2016-Dec2020.csv"
weather_data_df = pd.read_csv(csv_file2)
weather_data_df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [5]:
weather_data_df['date'] = pd.to_datetime(weather_data_df['StartTime(UTC)']).dt.date
weather_data_df['time'] = pd.to_datetime(weather_data_df['StartTime(UTC)']).dt.time
weather_data_df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,date,time
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-06,23:14:00
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07,04:14:00
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07,05:54:00
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08,05:34:00
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08,13:54:00


## Transform DataFrames

### Accident DataFrame

In [6]:
transformed_accident = accident_data_df[[ "date", "time", "City", "State","Start_Lat", "Start_Lng", "Description", 
                                         "Severity", "Weather_Condition", "Visibility(mi)", "Wind_Speed(mph)", 
                                         "Precipitation(in)"]]
transformed_accident.head()
#transformed_accident.count()

,date,time,City,State,Start_Lat,Start_Lng,Description,Severity,Weather_Condition,Visibility(mi),Wind_Speed(mph),Precipitation(in)
0,2019-05-21,08:29:55,Greenville,SC,34.808868,-82.269157,Accident on Tanner Rd at Pennbrooke Ln.,2,Fair,10.0,7.0,0.0
1,2019-10-07,17:43:09,Charlotte,NC,35.090080,-80.745560,Accident on Houston Branch Rd at Providence Br...,2,Cloudy,10.0,3.0,0.0
2,2020-12-13,21:53:00,Los Gatos,CA,37.145730,-121.985052,Stationary traffic on CA-17 from Summit Rd (CA...,2,Fair,10.0,6.0,0.0
3,2018-04-17,16:51:23,Carson City,NV,39.110390,-119.773781,Accident on US-395 Southbound at Topsy Ln.,2,Clear,10.0,4.6,NaN
4,2016-08-31,17:40:49,Fort Lauderdale,FL,26.102942,-80.265091,Accident on I-595 Westbound at Exit 4 / Pine I...,3,Overcast,10.0,13.8,NaN


In [7]:
#Rename columns
transformed_accident = transformed_accident.rename(columns = {"Start_Lat" : "lat",
                                                             "Start_Lng" : "lng"})

transformed_accident.head()

,date,time,City,State,lat,lng,Description,Severity,Weather_Condition,Visibility(mi),Wind_Speed(mph),Precipitation(in)
0,2019-05-21,08:29:55,Greenville,SC,34.808868,-82.269157,Accident on Tanner Rd at Pennbrooke Ln.,2,Fair,10.0,7.0,0.0
1,2019-10-07,17:43:09,Charlotte,NC,35.090080,-80.745560,Accident on Houston Branch Rd at Providence Br...,2,Cloudy,10.0,3.0,0.0
2,2020-12-13,21:53:00,Los Gatos,CA,37.145730,-121.985052,Stationary traffic on CA-17 from Summit Rd (CA...,2,Fair,10.0,6.0,0.0
3,2018-04-17,16:51:23,Carson City,NV,39.110390,-119.773781,Accident on US-395 Southbound at Topsy Ln.,2,Clear,10.0,4.6,NaN
4,2016-08-31,17:40:49,Fort Lauderdale,FL,26.102942,-80.265091,Accident on I-595 Westbound at Exit 4 / Pine I...,3,Overcast,10.0,13.8,NaN


In [8]:
# Clean the data by dropping duplicates
transformed_accident.drop_duplicates
transformed_accident.count()

date                 2906610
time                 2906610
City                 2906502
State                2906610
lat                  2906610
lng                  2906610
Description          2906610
Severity             2906610
Weather_Condition    2834759
Visibility(mi)       2834532
Wind_Speed(mph)      2599447
Precipitation(in)    1605284
dtype: int64

In [9]:
transformed_accident.sort_values(by = "State")
accident_ca = transformed_accident.loc[(transformed_accident["State"]) == "CA" ]
accident_ca

,date,time,City,State,lat,lng,Description,Severity,Weather_Condition,Visibility(mi),Wind_Speed(mph),Precipitation(in)
2,2020-12-13,21:53:00,Los Gatos,CA,37.145730,-121.985052,Stationary traffic on CA-17 from Summit Rd (CA...,2,Fair,10.0,6.0,0.0
7,2019-12-21,23:59:00,Whittier,CA,34.034017,-118.026972,At CA-60/Pomona Fwy - Accident.,2,Fair,10.0,5.0,0.0
9,2019-01-30,08:44:18,Valencia,CA,34.426330,-118.585100,At Magic Mountain Pky - Accident. Hard shoulde...,2,Mostly Cloudy,5.0,5.0,0.0
16,2016-07-09,14:54:01,Los Angeles,CA,34.103172,-118.249969,Accident on CA-2 Southbound at Exit 13 I-5.,2,Clear,10.0,3.5,NaN
17,2020-11-21,16:32:00,Fontana,CA,34.186595,-117.439427,Incident on SIERRA AVE near LYTLE CREEK RD Exp...,2,Mostly Cloudy,9.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2906586,2019-10-25,08:00:41,Bell Gardens,CA,33.981899,-118.191948,Accident on Randolph St at Otis Ave.,2,Fair,10.0,0.0,0.0
2906589,2017-08-24,07:53:55,La Crescenta,CA,34.231285,-118.266190,Accident on I-210 Eastbound at Lowell Ave.,3,Haze,5.0,3.5,NaN
2906590,2018-04-18,17:16:44,Hayward,CA,37.676400,-122.109800,At Meekland Ave - Accident.,3,Scattered Clouds,10.0,8.1,NaN
2906599,2020-02-24,11:34:00,Compton,CA,33.888992,-118.188797,At Alondra Blvd - Accident.,2,Fair,10.0,0.0,0.0


In [10]:
ca_year = accident_ca[accident_ca["date"].isin(pd.date_range('2019-01-01', '2019-12-31'))]
new_acc_ca = ca_year.set_index(['date'])
new_acc_ca.sort_index(inplace=True, ascending=True)
new_acc_ca

,time,City,State,lat,lng,Description,Severity,Weather_Condition,Visibility(mi),Wind_Speed(mph),Precipitation(in)
date,,,,,,,,,,,
2019-01-21,10:25:14,Los Angeles,CA,34.049473,-118.448677,#2 lane blocked due to accident on I-405 South...,3,Partly Cloudy,10.0,10.4,NaN
2019-01-21,14:06:57,Los Angeles,CA,33.996487,-118.280670,Two lanes blocked due to accident on I-110 Sou...,3,Clear,10.0,5.8,NaN
2019-01-21,15:44:34,Fullerton,CA,33.856384,-117.973167,Accident on Ca-91 Hov Ln Westbound at I-5 Hov ...,2,Clear,10.0,17.3,NaN
2019-01-21,18:36:40,Bellflower,CA,33.877190,-118.133850,Lane blocked due to accident on CA-91 Westboun...,2,Clear,10.0,19.6,NaN
2019-01-21,05:06:20,Claremont,CA,34.120579,-117.736328,Right hand shoulder blocked due to accident on...,2,Partly Cloudy,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2019-11-28,04:04:04,San Diego,CA,32.778931,-117.061584,#3 #4 and #5 lane blocked due to recovery work...,3,Cloudy,10.0,8.0,0.00
2019-11-28,08:13:39,Vallejo,CA,38.137341,-122.257080,Accident on entry ramp to CA-37 Eastbound from...,2,Light Snow,9.0,13.0,0.00
2019-11-28,00:40:09,Oakland,CA,37.826923,-122.300758,Accident on I-880 Southbound at I-80.,2,Cloudy,10.0,9.0,0.00


### Weather DataFrame

In [11]:
transformed_weather = weather_data_df[["date", "time", "LocationLat", "LocationLng", "Type", "Severity", "State"]]
transformed_weather.head()
transformed_weather.count()

date           6274206
time           6274206
LocationLat    6274206
LocationLng    6274206
Type           6274206
Severity       6274206
State          6274206
dtype: int64

In [12]:
#Rename columns
transformed_weather = transformed_weather.rename(columns = {"LocationLat" : "lat",
                                                            "LocationLng" : "lng"})

transformed_weather.head()

,date,time,lat,lng,Type,Severity,State
0,2016-01-06,23:14:00,38.0972,-106.1689,Snow,Light,CO
1,2016-01-07,04:14:00,38.0972,-106.1689,Snow,Light,CO
2,2016-01-07,05:54:00,38.0972,-106.1689,Snow,Light,CO
3,2016-01-08,05:34:00,38.0972,-106.1689,Snow,Light,CO
4,2016-01-08,13:54:00,38.0972,-106.1689,Snow,Light,CO


In [13]:
# Clean the data by dropping duplicates
transformed_weather.drop_duplicates
transformed_weather.count()

date        6274206
time        6274206
lat         6274206
lng         6274206
Type        6274206
Severity    6274206
State       6274206
dtype: int64

In [14]:
transformed_weather.sort_values(by = "State")
weather_ca = transformed_weather.loc[(transformed_weather["State"]) == "CA" ]
weather_ca

,date,time,lat,lng,Type,Severity,State
86134,2016-01-02,00:53:00,36.9394,-121.7900,Fog,Moderate,CA
86135,2016-01-04,02:53:00,36.9394,-121.7900,Rain,Light,CA
86136,2016-01-04,11:53:00,36.9394,-121.7900,Rain,Light,CA
86137,2016-01-05,06:53:00,36.9394,-121.7900,Rain,Light,CA
86138,2016-01-05,12:48:00,36.9394,-121.7900,Rain,Moderate,CA
...,...,...,...,...,...,...,...
6259341,2020-05-30,16:55:00,40.0167,-124.0667,Precipitation,UNK,CA
6259342,2020-08-21,23:35:00,40.0167,-124.0667,Cold,Severe,CA
6259343,2020-11-13,16:55:00,40.0167,-124.0667,Precipitation,UNK,CA
6259344,2020-12-17,00:35:00,40.0167,-124.0667,Precipitation,UNK,CA


In [15]:
weather_year = weather_ca[weather_ca["date"].isin(pd.date_range('2019-01-01', '2019-12-31'))]
new_weather_ca = weather_year.set_index(['date'])
new_weather_ca.sort_index(inplace=True, ascending=True)
new_weather_ca

,time,lat,lng,Type,Severity,State
date,,,,,,
2019-01-21,01:32:00,36.9394,-121.7900,Rain,Light,CA
2019-01-21,07:57:00,38.8984,-119.9962,Snow,Light,CA
2019-01-21,06:58:00,38.8984,-119.9962,Snow,Light,CA
2019-01-21,06:53:00,38.8984,-119.9962,Snow,Moderate,CA
2019-01-21,04:01:00,38.8984,-119.9962,Fog,Severe,CA
...,...,...,...,...,...,...
2019-11-28,08:09:00,33.9000,-117.2500,Rain,Light,CA
2019-11-28,09:38:00,33.9000,-117.2500,Rain,Light,CA
2019-11-28,20:51:00,34.3667,-118.5667,Rain,Light,CA


## Create Database Connection

In [ ]:
# configure the connection string
rds_connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# confirm the connection string
print(rds_connection_string)
        
# connect to the database
engine = create_engine(rds_connection_string)
conn = engine.connect()